# IMPORT

In [24]:
import random
import requests
import os
import json
import time

## Set UP AZURE API

In [43]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time
global subscription_key
global endpoint
subscription_key = "<Removed>"
endpoint = "https://azurehackforaccessibility.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
local_image_path = "/Users/kunal/Documents/AzureCompetition/Testing_img_NEW290_.jpg"

## Set-up Selenium

In [26]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\kunal\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [28]:
driver.quit()

## Paths Managment

In [48]:
global filePaths 
filePaths = "/Users/kunal/Documents/AzureCompetition"

# Street View API Pictures

In [49]:
def callAPI(heading, lat, long, pitch, fov, radius,source):
    #print("Calls the streetview API")
    pic_base = 'https://maps.googleapis.com/maps/api/streetview?'
    api_key = '<Removed>'
    size = '640x640'
    lat = lat
    long = long
    radius=radius
    source=source
    picLINK = pic_base+"size="+str(size)+"&"+"location="+str(lat)+","+str(long)+"&"+"heading="+str(heading)+"&"+"pitch="+str(pitch)+"&"+"fov="+str(fov)+"&"+"radius="+str(radius)+"&"+"source="+str(source)+"&"+"key="+str(api_key)
    pic_response = requests.get(picLINK) #This costs Money to run this cell as it does call the API
    return pic_response

In [50]:
def downloadpic(heading, lat, long, pitch, fov,radius,source, printINFO = False, display = False):
    #print("Calls the streetview API and saves it")
    pic_response = callAPI(heading, lat, long, pitch, fov, radius,source)
    if printINFO == True:
        for key, value in pic_response.headers.items():
            print(f"{key}: {value}")
    if not pic_response.ok:
        for key, value in pic_response.headers.items():
            print(f"{key}: {value}")
        raise Exception("Picture doesn't exist and there is an error")
    filename = "img_H"+str(heading)+"_P"+str(pitch)+"_F"+str(fov)+"_"+str(lat)+"_"+str(long) + ".jpg"
    with open(filename, 'wb') as file:
        file.write(pic_response.content)
    pic_response.close()
    print("Saved "+ filename)
    if display == True:
        # using matpltolib to display the image
        import matplotlib.pyplot as plt
        import matplotlib.image as mpimg
        plt.figure(figsize=(10, 10))
        img=mpimg.imread(fileName)
        imgplot = plt.imshow(img)
        plt.show()
    return filename

In [51]:
def metaDataforPIC(lat, long, heading, pitch, fov, radius, source):
    size = '640x640'
    api_key = '<Removed>'
    #print("Using same data from street view API it returns data like detailed LAT long and pano ID and other things")
    meta_base = 'https://maps.googleapis.com/maps/api/streetview/metadata?'
    metaDataLink = meta_base+"size="+str(size)+"&"+"location="+str(lat)+","+str(long)+"&"+"heading="+str(heading)+"&"+"pitch="+str(pitch)+"&"+"fov="+str(fov)+"&"+"radius="+str(radius)+"&"+"source="+str(source)+"&"+"key="+str(api_key)
    meta_response = requests.get(metaDataLink)
    return meta_response.json()

# Lat Long Managment

In [84]:
def latlongToAddy(lat,long):
    #print("Opens Selenium and converts Lat long number to address and returns string address")
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get("https://www.latlong.net/Show-Latitude-Longitude.html")
    lat = "/html/body/main/div[2]/div[1]/form/div[1]/div[1]/input"
    long = "/html/body/main/div[2]/div[1]/form/div[1]/div[2]/input"
    convert = "/html/body/main/div[2]/div[1]/form/div[1]/div[3]/button"
    answer = "/html/body/main/div[2]/div[1]/form/div[2]"
    latNum = str(lat)
    longNum = str(long)
    driver.find_element_by_xpath(lat).send_keys(Keys.CONTROL, 'a')
    driver.find_element_by_xpath(lat).send_keys(latNum)
    driver.find_element_by_xpath(long).send_keys(Keys.CONTROL, 'a')
    time.sleep(0.5)
    driver.find_element_by_xpath(long).send_keys(longNum)
    driver.find_element_by_xpath(convert).send_keys(Keys.ENTER)
    addy = driver.find_element_by_xpath(answer).text
    driver.quit()
    return addy

In [53]:
def addytoLatLong(city, country):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="my_user_agent")
    loc = geolocator.geocode(city+','+ country)
    latNumCalculated = loc.latitude
    longNumCalculated = loc.longitude
    return latNumCalculated, longNumCalculated

# Azure API Managment

In [54]:
def tagIMG(local_image_path):
    #print("Calls Azure API for tagging image and returns \n1)2D array [tag name, confid, hint] \n2)width \n3) height \n4)format \n5)requestID")
    local_image = open(local_image_path, "rb")
    tags_result_local = computervision_client.tag_image_in_stream(local_image)
    returnArray = []
    for tag in tags_result_local.tags:
        returnArray.append([tag.name, tag.confidence * 100, tag.hint])
        #print("\t'{}' with confidence {:.2f}% and with hints of {}".format(tag.name, tag.confidence * 100, tag.hint))
    #print("== MetaData ==")
    #print("\tWidth: "+str(tags_result_local.metadata.width))
    #print("\tHeight: "+str(tags_result_local.metadata.height))
    #print("\tFormat: "+tags_result_local.metadata.format)
    #print("\tRequest ID: "+tags_result_local.request_id)
    return returnArray, tags_result_local.metadata.width, tags_result_local.metadata.height, tags_result_local.metadata.format, tags_result_local.request_id

In [55]:
def describeIMG(local_image_path):
    #print("Calls Azure API for describition of image and returns \n1)2D array [caption, confid] \n2)tags in array \n3)requestID")
    local_image = open(local_image_path, "rb")
    description_result = computervision_client.describe_image_in_stream(local_image)
    #print("== Description ==")
    #if (len(description_result.captions) == 0):
        #print("No description detected.")
    #else:
    returnArrayCaption = []
    for caption in description_result.captions:
        #print("\t'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))
        returnArrayCaption.append([caption.text, caption.confidence * 100])
    #print("== TAGS ==")
    returnArrayTags = []
    for i in description_result.tags:
        returnArrayTags.append(i)
        #print("\tTags: {}".format(i))
    #print("REQUEST ID: {}".format(description_result.request_id))
    return returnArrayCaption, returnArrayTags, description_result.request_id

In [56]:
def getObjs(local_image_path):
    #print("Calls Azure API for ojects in image and returns \n1)2D Array [X,Y,W,H] \n2)RequestID")
    local_image_path_objects = local_image_path
    local_image_objects = open(local_image_path_objects, "rb")
    detect_objects_results_local = computervision_client.detect_objects_in_stream(local_image_objects)
    #print("Detecting objects in local image:")
    #if len(detect_objects_results_local.objects) == 0:
        #print("No objects detected.")
    #else:
    returnArrayOBJ = []
    for object in detect_objects_results_local.objects:
        returnArrayOBJ.append([object.rectangle.x, object.rectangle.y, object.rectangle.w, object.rectangle.h])
            #print("object at location {}, {}, {}, {}".format( \
            #object.rectangle.x, object.rectangle.x + object.rectangle.w, \
            #object.rectangle.y, object.rectangle.y + object.rectangle.h))
    #print("REQUEST ID: {}".format(detect_objects_results_local.request_id))
    return returnArrayOBJ, detect_objects_results_local.request_id

In [57]:
def intreast(local_image_path):
    #print("Calls Azure API for intreast in image and returns \n1)Array [X,Y,W,H] \n2)RequestID")
    local_image = open(local_image_path, "rb")
    area_interest = computervision_client.get_area_of_interest_in_stream(local_image)

    #print("Area (X,Y,W,H): {},{},{},{}".format(area_interest.area_of_interest.x, area_interest.area_of_interest.y, 
                                               #area_interest.area_of_interest.w, area_interest.area_of_interest.h))
    #print("REQUEST ID: {}".format(area_interest.request_id))
    return [area_interest.area_of_interest.x, area_interest.area_of_interest.y, area_interest.area_of_interest.w, area_interest.area_of_interest.h], area_interest.request_id

In [58]:
def anaylsis(local_image_path):
    #print("Calls Azure API for anay;sos in image and returns \n1)2D Array [name, score] \n2)RequestID")
    local_image = open(local_image_path, "rb")
    categorize_results_local = computervision_client.analyze_image_in_stream(local_image)
    returnArrayCat = []
    for i in categorize_results_local.categories:
        #print("\tCatogory: {} == Score: {}".format(i.name, i.score))
        returnArrayCat.append([i.name, i.score])
        #if i.detail != None:
            #for j in i.detail:
                #print(j)
                #for r in j.celebrities:
                    #print("\t\tCelebrity: {} == Score: {}".format(r.name, r.confidence))
                    #print("Face rectangle")
                #for r in j.landmarks:
                    #print("\t\tLandMark: {} == Score: {}".format(r.name, r.confidence))
                    #print("Face rectangle")
    #print("REQUEST ID: {}".format(categorize_results_local.request_id))
    return returnArrayCat, categorize_results_local.request_id

# EncodeIMG

In [59]:
import base64

def getEncodImg(filename):
    with open(filename, 'rb') as binary_file:
        binary_file_data = binary_file.read()
        base64_encoded_data = base64.b64encode(binary_file_data)
        base64_message = base64_encoded_data.decode('utf-8')
    return base64_message

# Calling it All - Need to change

In [60]:
def createJSON(direction, pathIMG, pitch, fov):
    imgTagging1Name = ""
    imgTagging1Confid = ""
    imgTagging1Hint = ""
    imgTagging2Name = ""
    imgTagging2Confid = ""
    imgTagging2Hint = ""
    imgTagging3Name = ""
    imgTagging3Confid = ""
    imgTagging3Hint = ""
    imgTagging4Name = ""
    imgTagging4Confid = ""
    imgTagging4Hint = ""
    imgTagging5Name = ""
    imgTagging5Confid = ""
    imgTagging5Hint = ""
    imgTagging6Name = ""
    imgTagging6Confid = ""
    imgTagging6Hint = ""
    imgTagging7Name = ""
    imgTagging7Confid = ""
    imgTagging7Hint = ""
    imgTagging8Name = ""
    imgTagging8Confid = ""
    imgTagging8Hint = ""
    imgTagging9Name = ""
    imgTagging9Confid = ""
    imgTagging9Hint = ""
    imgTagging10Name = ""
    imgTagging10Confid = ""
    imgTagging10Hint = ""
    anaylsisImg1Name = ""
    anaylsisImg1Score = ""
    anaylsisImg2Name = ""
    anaylsisImg2Score = ""
    anaylsisImg3Name = ""
    anaylsisImg3Score = ""
    anaylsisImg4Name = ""
    anaylsisImg4Score = ""
    anaylsisImg5Name = ""
    anaylsisImg5Score = ""
    
    objectboundingBoxOBJ_1 = []
    objectboundingBoxOBJ_2 = []
    objsArray , objectsRequestID = getObjs(pathIMG)
    if len(objsArray) == 0:
        objectboundingBoxOBJ_1 = [0,0,0,0]
        objectboundingBoxOBJ_2 = [0,0,0,0]
    elif len(objsArray) == 1:
        objectboundingBoxOBJ_1 = objsArray[0]
        objectboundingBoxOBJ_2 = [0,0,0,0]
    else: 
        objectboundingBoxOBJ_1 = objsArray[0]
        objectboundingBoxOBJ_2 = objsArray[1]
    #--------------------------------------------------------
    encodedImg1 = getEncodImg(pathIMG)
    pitchImg1 = pitch
    fovImg1 = fov
    tagsArray,metaWidth,metaHeight,metaFormat,imgTaggingRequestId = tagIMG(pathIMG)  
    try:
        imgTagging1Name = tagsArray[0][0]
        imgTagging1Confid = tagsArray[0][1]
        imgTagging1Hint = tagsArray[0][2]
    except:
        pass
    try:
        imgTagging2Name = tagsArray[1][0]
        imgTagging2Confid = tagsArray[1][1]
        imgTagging2Hint = tagsArray[1][2]
    except:
        pass
    try:
        imgTagging3Name = tagsArray[2][0]
        imgTagging3Confid = tagsArray[2][1]
        imgTagging3Hint = tagsArray[2][2]
    except:
        pass
    try:
        imgTagging4Name = tagsArray[3][0]
        imgTagging4Confid = tagsArray[3][1]
        imgTagging4Hint = tagsArray[3][2]
    except:
        pass
    try:
        imgTagging5Name = tagsArray[4][0]
        imgTagging5Confid = tagsArray[4][1]
        imgTagging5Hint = tagsArray[4][2]
    except:
        pass
    try:
        imgTagging6Name = tagsArray[5][0]
        imgTagging6Confid = tagsArray[5][1]
        imgTagging6Hint = tagsArray[5][2]
    except:
        pass
    try:
        imgTagging7Name = tagsArray[6][0]
        imgTagging7Confid = tagsArray[6][1]
        imgTagging7Hint = tagsArray[6][2]
    except:
        pass
    try:
        imgTagging8Name = tagsArray[7][0]
        imgTagging8Confid = tagsArray[7][1]
        imgTagging8Hint = tagsArray[7][2]
    except:
        pass
    try:
        imgTagging9Name = tagsArray[8][0]
        imgTagging9Confid = tagsArray[8][1]
        imgTagging9Hint = tagsArray[8][2]
    except:
        pass
    try:
        imgTagging10Name = tagsArray[9][0]
        imgTagging10Confid = tagsArray[9][1]
        imgTagging10Hint = tagsArray[9][2]
    except:
        pass
    
    descArray, describeTagsArray, describeImgRequestID = describeIMG(filenameetst)
    describeImgtext = descArray[0][0]
    describeImgcondif = descArray[0][1]
    
    areaIntreastArray, areaIntreastRequestID = intreast(filenameetst)
    
    anaylsisArray,anaylsisRequestID= anaylsis(filenameetst)
    try:
        anaylsisImg1Name = anaylsisArray[0][0]
        anaylsisImg1Score = anaylsisArray[0][1]
    except:
        pass
    try:
        anaylsisImg2Name = anaylsisArray[1][0]
        anaylsisImg2Score = anaylsisArray[1][1]
    except:
        pass
    try:
        anaylsisImg3Name = anaylsisArray[2][0]
        anaylsisImg3Score = anaylsisArray[2][1]
    except:
        pass
    try:
        anaylsisImg4Name = anaylsisArray[3][0]
        anaylsisImg4Score = anaylsisArray[3][1]
    except:
        pass
    try:
        anaylsisImg5Name = anaylsisArray[4][0]
        anaylsisImg5Score = anaylsisArray[4][1]
    except:
        pass
    baseDict = {
       "Direction_Degrees": direction,
       "EncodedImg": str(encodedImg1),
       "Pitch": pitchImg1,
       "ZoomAndFov": fovImg1,
       "CalculatedOutputAzure" : {
           "ImageTagging": {
               "Tags": {
                        imgTagging1Name: {
                                         "Confidence": imgTagging1Confid,
                                         "Hint": imgTagging1Hint
                        }, imgTagging2Name: {
                                         "Confidence": imgTagging2Confid,
                                         "Hint": imgTagging2Hint
                        }, imgTagging3Name: {
                                         "Confidence": imgTagging3Confid,
                                         "Hint": imgTagging3Hint
                        }, imgTagging4Name: {
                                         "Confidence": imgTagging4Confid,
                                         "Hint": imgTagging4Hint
                        }, imgTagging5Name: {
                                         "Confidence": imgTagging5Confid,
                                         "Hint": imgTagging5Hint
                        }, imgTagging6Name: {
                                         "Confidence": imgTagging6Confid,
                                         "Hint": imgTagging6Hint
                        }, imgTagging7Name: {
                                         "Confidence": imgTagging7Confid,
                                         "Hint": imgTagging7Hint
                        }, imgTagging8Name: {
                                         "Confidence": imgTagging8Confid,
                                         "Hint": imgTagging8Hint
                        }, imgTagging9Name: {
                                         "Confidence": imgTagging9Confid,
                                         "Hint": imgTagging9Hint
                        }, imgTagging10Name: {
                                         "Confidence": imgTagging10Confid,
                                         "Hint": imgTagging10Hint
                        }, 
                    },
                "RequestID": str(imgTaggingRequestId)
               }, "DescribeImg": {
                    "Description": {
                        "Text": str(describeImgtext),
                        "Confidence": describeImgcondif
                    },
                    "Tags": describeTagsArray,
                    "RequestID": str(describeImgRequestID)
               }, "AnalysisImg": {
                    "Analysis": {
                        str(anaylsisImg1Name): {
                            "Score": anaylsisImg1Score
                        },str(anaylsisImg2Name): {
                            "Score": anaylsisImg2Score
                        },str(anaylsisImg3Name): {
                            "Score": anaylsisImg3Score
                        },str(anaylsisImg4Name): {
                            "Score": anaylsisImg4Score
                        },str(anaylsisImg5Name): {
                            "Score": anaylsisImg5Score
                        }

                    },
                    "RequestID": str(anaylsisRequestID)
                }, "ObjectsImg": {
                    "Objects": {
                        "BoundingBox_XYWH_1": objectboundingBoxOBJ_1,
                        "BoundingBox_XYWH_2": objectboundingBoxOBJ_2
                    },
                    "RequestID": objectsRequestID
                },"AreaIntreast": {
                    "IntreastLocation_XYWH": areaIntreastArray,
                    "RequestID": str(areaIntreastRequestID)
                },
                "MetaData": {
                    "Width": str(metaWidth),
                    "Height": str(metaHeight),
                    "Format": str(metaFormat)
                }
           }
       }
    return baseDict

In [75]:
def savePics(lat,long):
    os.chdir(filePaths)
    os.mkdir("IMG_" + str(lat) + "_" + str(long))
    os.chdir("IMG_" + str(lat) + "_" + str(long))
    print("Created: "+ "IMGS_" + str(lat) + "_" + str(long))
    headings = [0,90,180,270]
    pitch = 0
    fov = 75
    radius = 50
    source ="outdoor"
    fileNamesLst = []
    dictionarysALL = []
    for dire in headings:
        name = downloadpic(dire, lat, long, pitch, fov,radius,source)
        fileNamesLst.append(name)
        dictionarysALL.append(createJSON(dire, name, pitch, fov))

    metaResult = metaDataforPIC(lat, long, 0, pitch, fov, radius, source)
    
    titleName = "Location_" + str(lat) + "_" + str(long)
    panoID = metaResult['pano_id']
    address = latlongToAddy(lat,long)
    
    
    
    dictionary = {titleName: 
                  {"LocationInformation" : 
                   {"PanoID": str(panoID),
                    "Latitude": lat,
                    "Longitude": long,
                    "Address": str(address) 
                   }, 
                    "Img1_0" : dictionarysALL[0],
                    "Img2_90" : dictionarysALL[1],
                    "Img3_180" : dictionarysALL[2],
                    "Img4_270" : dictionarysALL[3]
                  }
                }

    jsonString = json.dumps(dictionary, indent=4)
    
    print(jsonString)
    with open('data.json', 'w') as f:
        json.dump(jsonString, f)
    return jsonString
    os.chdir(filePaths)

In [92]:
lat = 33.7748583
long=-84.3931146

In [93]:
JSON = savePics(lat, long)

Created: IMGS_33.7748583_-84.3931146
Saved img_H0_P0_F75_33.7748583_-84.3931146.jpg


ComputerVisionErrorException: Operation returned an invalid status code 'Too Many Requests'

In [ ]:
db.test.insert_one(JSON)

# econdiong


In [87]:
import pymongo
import requests
from pymongo import MongoClient  
import json

In [88]:
uri = "<Removed>"
client = pymongo.MongoClient(uri)
dblist = client.list_database_names()
if "Azure-Hack" in dblist:
    print("The database exists.")
    db = client["Azure-Hack"]
db.test

The database exists.


Collection(Database(MongoClient(host=['cosmos-az.mongo.cosmos.azure.com:10255'], document_class=dict, tz_aware=False, connect=True, ssl=True, replicaset='globaldb', retrywrites=False, maxidletimems=120000, appname='@cosmos-az@'), 'Azure-Hack'), 'test')

In [89]:
# Created or Switched to collection
Collection = db["data"] 

Collection
#db.test.insert_one({'x': 1})

Collection(Database(MongoClient(host=['cosmos-az.mongo.cosmos.azure.com:10255'], document_class=dict, tz_aware=False, connect=True, ssl=True, replicaset='globaldb', retrywrites=False, maxidletimems=120000, appname='@cosmos-az@'), 'Azure-Hack'), 'data')

# Ignore Everything Below

In [78]:
lat = 3.255234534
long = 23.45345345
titleName = "Location_" + str(lat) + "_" + str(long)
panoID = ""
address = ""
encodedImg1 = ""
pitchImg1 = ""
fovImg1 = ""
#=================================
imgTagging1Name = ""
imgTagging1Confid = ""
imgTagging1Hint = ""
#--------------------
imgTagging2Name = ""
imgTagging2Confid = ""
imgTagging2Hint = ""
#--------------------
imgTagging3Name = ""
imgTagging3Confid = ""
imgTagging3Hint = ""
#--------------------
imgTagging4Name = ""
imgTagging4Confid = ""
imgTagging4Hint = ""
#--------------------
imgTagging5Name = ""
imgTagging5Confid = ""
imgTagging5Hint = ""
#--------------------
imgTagging6Name = ""
imgTagging6Confid = ""
imgTagging6Hint = ""
#--------------------
imgTagging7Name = ""
imgTagging7Confid = ""
imgTagging7Hint = ""
#--------------------
imgTagging8Name = ""
imgTagging8Confid = ""
imgTagging8Hint = ""
#--------------------
imgTagging9Name = ""
imgTagging9Confid = ""
imgTagging9Hint = ""
#--------------------
imgTagging10Name = ""
imgTagging10Confid = ""
imgTagging10Hint = ""
imgTaggingRequestId = ""
#=================================
describeImgtext = ""
describeImgcondif = ""
describeTagsArray = []
describeImgRequestID = ""
#===============================
anaylsisImg1Name = ""
anaylsisImg1Score = ""
#--------------------
anaylsisImg2Name = ""
anaylsisImg2Score = ""
#--------------------
anaylsisImg3Name = ""
anaylsisImg3Score = ""
#--------------------
anaylsisImg4Name = ""
anaylsisImg4Score = ""
#--------------------
anaylsisImg5Name = ""
anaylsisImg5Score = ""
#--------------------
anaylsisRequestID = ""
#===============================
objectboundingBoxOBJ_1 = []
objectboundingBoxOBJ_2 = []
objectsRequestID = ""
#===============================
areaIntreastArray = []
areaIntreastRequestID = ""
#===============================
metaWidth = ""
metaWeight = ""
metaFormat = ""

In [13]:
lat = ""

In [17]:
dict1 = {}
dict2 = {}
dict3 = {}
dict4 = {}

In [20]:


dictionary = {titleName: 
              {"LocationInformation" : 
               {"PanoID": str(panoID),
                "Latitude": str(lat),
                "Longitude": str(lat),
                "Address": str(address) 
               }, 
                "Img1_0" : dict1,
                "Img2_90" : dict1,
                "Img3_180" : dict1,
                "Img4_270" : dict1
              }
            }
              
#jsonString = json.dumps(dictionary, indent=4)
#print(jsonString)

{
    "": {
        "LocationInformation": {
            "PanoID": "",
            "Latitude": "",
            "Longitude": "",
            "Address": ""
        },
        "Img1_0": {},
        "Img2_90": {},
        "Img3_180": {},
        "Img4_270": {}
    }
}


In [79]:
import json

dictionary = {titleName: 
              {"LocationInformation" : 
               {"PanoID": str(panoID),
                "Latitude": str(lat),
                "Longitude": str(lat),
                "Address": str(address) 
               }, "Img1_0" : {
                   "Direction_Degrees": str(0),
                   "EncodedImg": str(encodedImg1),
                   "Pitch": str(pitchImg1),
                   "ZoomAndFov": str(fovImg1),
                   "CalculatedOutputAzure" : {
                       "ImageTagging": {
                           "Tags": {
                                    imgTagging1Name: {
                                                     "Confidence": imgTagging1Confid,
                                                     "Hint": imgTagging1Hint
                                    }, imgTagging2Name: {
                                                     "Confidence": imgTagging2Confid,
                                                     "Hint": imgTagging2Hint
                                    }, imgTagging3Name: {
                                                     "Confidence": imgTagging3Confid,
                                                     "Hint": imgTagging3Hint
                                    }, imgTagging4Name: {
                                                     "Confidence": imgTagging4Confid,
                                                     "Hint": imgTagging4Hint
                                    }, imgTagging5Name: {
                                                     "Confidence": imgTagging5Confid,
                                                     "Hint": imgTagging5Hint
                                    }, imgTagging6Name: {
                                                     "Confidence": imgTagging6Confid,
                                                     "Hint": imgTagging6Hint
                                    }, imgTagging7Name: {
                                                     "Confidence": imgTagging7Confid,
                                                     "Hint": imgTagging7Hint
                                    }, imgTagging8Name: {
                                                     "Confidence": imgTagging8Confid,
                                                     "Hint": imgTagging8Hint
                                    }, imgTagging9Name: {
                                                     "Confidence": imgTagging9Confid,
                                                     "Hint": imgTagging9Hint
                                    }, imgTagging10Name: {
                                                     "Confidence": imgTagging10Confid,
                                                     "Hint": imgTagging10Hint
                                    }, 
                                },
                            "RequestID": str(imgTaggingRequestId)
                           }, "DescribeImg": {
                                "Description": {
                                    "Text": str(describeImgtext),
                                    "Confidence": str(describeImgcondif)
                                },
                                "Tags": describeTagsArray,
                                "RequestID": str(describeImgRequestID)
                           }, "AnalysisImg": {
                                "Analysis": {
                                    str(anaylsisImg1Name): {
                                        "Score": anaylsisImg1Score
                                    },str(anaylsisImg2Name): {
                                        "Score": anaylsisImg2Score
                                    },str(anaylsisImg3Name): {
                                        "Score": anaylsisImg3Score
                                    },str(anaylsisImg4Name): {
                                        "Score": anaylsisImg4Score
                                    },str(anaylsisImg5Name): {
                                        "Score": anaylsisImg5Score
                                    }
                                    
                                },
                                "RequestID": str(anaylsisRequestID)
                            }, "ObjectsImg": {
                                "Objects": {
                                    "BoundingBox_XYWH_1": objectboundingBoxOBJ_1,
                                    "BoundingBox_XYWH_2": objectboundingBoxOBJ_2
                                },
                                "RequestID": objectsRequestID
                            },"AreaIntreast": {
                                "IntreastLocation_XYWH": areaIntreastArray,
                                "RequestID": str(areaIntreastRequestID)
                            },
                            "MetaData": {
                                "Width": str(metaWidth),
                                "Height": str(metaHeight),
                                "Format": str(metaFormat)
                            }
                       }
                   }
               }
              }
              
jsonString = json.dumps(dictionary, indent=4)
print(jsonString)

{
    "Location_3.255234534_23.45345345": {
        "LocationInformation": {
            "PanoID": "",
            "Latitude": "3.255234534",
            "Longitude": "3.255234534",
            "Address": ""
        },
        "Img1_0": {
            "Direction_Degrees": "0",
            "PicturePath": "",
            "Pitch": "",
            "ZoomAndFov": "",
            "CalculatedOutputAzure": {
                "ImageTagging": {
                    "Tags": {
                        "": {
                            "Confidence": "",
                            "Hint": ""
                        }
                    },
                    "RequestID": ""
                },
                "DescribeImg": {
                    "Description": {
                        "Text": "",
                        "Confidence": ""
                    },
                    "Tags": [],
                    "RequestID": ""
                },
                "AnalysisImg": {
                    "Anal

# JSON OUTPUT

In [82]:

			"AreaIntreast": {
				"IntreastLocation": {
					"X": str(lat),
					"Y": str(lat),
					"W": str(lat),
					"H": str(lat)
				},
				"RequestID": str(lat)
			},
			"MetaData": {
				"Width": str(lat),
				"Height": str(lat),
				"Format": str(lat)
			}

		}


	}

}
}

In [67]:
from pprint import pprint

In [81]:
print(dictNorm)

{'Location_33.775215_-84.396067': {'LocationInformation': {'PanoID': '33.775215', 'Latitude': '33.775215', 'Longitude': '33.775215', 'Address': '33.775215'}, 'Img1_0': {'Direction_Degrees': '33.775215', 'PicturePath': '33.775215', 'Pitch': '33.775215', 'ZoomAndFov': '33.775215', 'CalculatedOutputAzure': {'ContainsSideWalk': '33.775215', 'ImageTagging': {'Tags': {'33.775215': {'Confidence': '33.775215', 'Hint': '33.775215'}}, 'RequestID': '33.775215'}, 'DescribeImg': {'Description': {'Text': '33.775215', 'Confidence': '33.775215'}, 'Tags': {'33.775215': '33.775215'}, 'RequestID': '33.775215'}, 'AnalysisImg': {'Analysis': {'33.775215': {'Score': '33.775215'}}, 'RequestID': '33.775215'}, 'ObjectsImg': {'Objects': {'33.775215': {'X': '33.775215', 'Y': '33.775215', 'W': '33.775215', 'H': '33.775215'}}, 'RequestID': '33.775215'}, 'AreaIntreast': {'IntreastLocation': {'X': '33.775215', 'Y': '33.775215', 'W': '33.775215', 'H': '33.775215'}, 'RequestID': '33.775215'}, 'MetaData': {'Width': '33.

In [76]:
L = """{'Location_33.775215_-84.396067': {'LocationInformation': {'PanoID': '33.775215',
   'Latitude': '33.775215',
   'Longitude': '33.775215',
   'Address': '33.775215'},
  'Img1_0': {'Direction_Degrees': '33.775215',
   'PicturePath': '33.775215',
   'Pitch': '33.775215',
   'ZoomAndFov': '33.775215',
   'CalculatedOutputAzure': {'ContainsSideWalk': '33.775215',
    'ImageTagging': {'Tags': {'33.775215': {'Confidence': '33.775215',
       'Hint': '33.775215'}},
     'RequestID': '33.775215'},
    'DescribeImg': {'Description': {'Text': '33.775215',
      'Confidence': '33.775215'},
     'Tags': {'33.775215': '33.775215'},
     'RequestID': '33.775215'},
    'AnalysisImg': {'Analysis': {'33.775215': {'Score': '33.775215'}},
     'RequestID': '33.775215'},
    'ObjectsImg': {'Objects': {'33.775215': {'X': '33.775215',
       'Y': '33.775215',
       'W': '33.775215',
       'H': '33.775215'}},
     'RequestID': '33.775215'},
    'AreaIntreast': {'IntreastLocation': {'X': '33.775215',
      'Y': '33.775215',
      'W': '33.775215',
      'H': '33.775215'},
     'RequestID': '33.775215'},
    'MetaData': {'Width': '33.775215',
     'Height': '33.775215',
     'Format': '33.775215'}}}}}"""

In [77]:
import json

def is_json(myjson):
    try:
        json_object = json.loads(myjson)
    except ValueError as e:
        return False
    return True

In [79]:
is_json(L)

False